In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [1]:
!pip install datasets
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
# persian-nlp

from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import json

# Load the model and tokenizer
model_name = "persiannlp/mt5-small-parsinlu-translation_en_fa"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

def translate_to_persian(sentences):
    translations = []
    for sentence in sentences:
        # Tokenize the input sentence
        input_ids = tokenizer.encode(sentence, return_tensors="pt")
        # Generate translation
        outputs = model.generate(input_ids)
        # Decode the translation
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translated_text)
    return translations

# Load your dataset
with open("./content/mmm.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Iterate over your data and translate captions
for video_id, video_info in data["database"].items():
    for annotation in video_info["annotations"]:
        english_sentence = annotation["sentence"]
        persian_translation = translate_to_persian([english_sentence])[0]
        annotation["translated_sentence"] = persian_translation

# Save the updated dataset
with open("./content/translated_dataset-persiannlp.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


  if decoder is not None:



In [1]:
import json
import csv
from deep_translator import GoogleTranslator
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

json_file_path = "./content/trainval_input.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

def evaluate_similarity(original, translated):
    embeddings = model.encode([original, translated])
    similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
    return similarity

training_data = []
validation_data = []
evaluated_training_data = []
evaluated_validation_data = []

for video_id, video_info in data["database"].items():
    subset = video_info.get("subset", "unknown")
    annotations = video_info.get("annotations", [])

    for annotation in annotations:
        original_sentence = annotation["sentence"]

        translated_sentence = GoogleTranslator(source='en', target='fa').translate(original_sentence)

        similarity_score = evaluate_similarity(original_sentence, translated_sentence)
        new_id = f"{video_id}_{annotation['id']}"

        translated_annotation = {
            "id": new_id,
            "sentence": translated_sentence
        }

        evaluated_annotation = {
            "id": new_id,
            "original_sentence": original_sentence,
            "translated_sentence": translated_sentence,
            "similarity_score": similarity_score
        }

        if subset == "training":
            training_data.append(translated_annotation)
            evaluated_training_data.append(evaluated_annotation)
        elif subset == "validation":
            validation_data.append(translated_annotation)
            evaluated_validation_data.append(evaluated_annotation)

translated_training_output_path = "./content/translated_training_dataset.csv"
translated_validation_output_path = "./content/translated_validation_dataset.csv"
evaluated_training_output_path = "./content/evaluated_training_dataset.csv"
evaluated_validation_output_path = "./content/evaluated_validation_dataset.csv"

with open(translated_training_output_path, mode="w", newline='', encoding="utf-8") as train_file:
    writer = csv.DictWriter(train_file, fieldnames=["id", "sentence"])
    writer.writeheader()
    for row in training_data:
        writer.writerow(row)

with open(translated_validation_output_path, mode="w", newline='', encoding="utf-8") as val_file:
    writer = csv.DictWriter(val_file, fieldnames=["id", "sentence"])
    writer.writeheader()
    for row in validation_data:
        writer.writerow(row)

with open(evaluated_training_output_path, mode="w", newline='', encoding="utf-8") as train_eval_file:
    writer = csv.DictWriter(train_eval_file, fieldnames=["id", "original_sentence", "translated_sentence", "similarity_score"])
    writer.writeheader()
    for row in evaluated_training_data:
        writer.writerow(row)

with open(evaluated_validation_output_path, mode="w", newline='', encoding="utf-8") as val_eval_file:
    writer = csv.DictWriter(val_eval_file, fieldnames=["id", "original_sentence", "translated_sentence", "similarity_score"])
    writer.writeheader()
    for row in evaluated_validation_data:
        writer.writerow(row)

print("Translation and similarity evaluation completed.")
print(f"Translated training data saved to {translated_training_output_path}")
print(f"Translated validation data saved to {translated_validation_output_path}")
print(f"Evaluated training data saved to {evaluated_training_output_path}")
print(f"Evaluated validation data saved to {evaluated_validation_output_path}")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Translation and similarity evaluation completed.
Translated training data saved to ./content/translated_training_dataset.csv
Translated validation data saved to ./content/translated_validation_dataset.csv
Evaluated training data saved to ./content/evaluated_training_dataset.csv
Evaluated validation data saved to ./content/evaluated_validation_dataset.csv


# Cutting video to ranges

In [2]:
!pip install moviepy

In [3]:
import json
from moviepy.video.io.VideoFileClip import VideoFileClip
import os

json_file_path = "./content/trainval_input.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

video_folder = "./content/videos/"

output_dir = "./content/video_segments/"
os.makedirs(output_dir, exist_ok=True)

for video_id, video_info in data["database"].items():
    video_path = os.path.join(video_folder, f"{video_id}.mp4")

    if not os.path.exists(video_path):
        print(f"Video file not found for {video_id}. Skipping...")
        continue

    print(f"Processing video: {video_id}")
    video = VideoFileClip(video_path)
    annotations = video_info["annotations"]

    for annotation in annotations:
        start, end = annotation["segment"]
        segment_id = annotation["id"]

        output_file = os.path.join(output_dir, f"{video_id}_{segment_id}.mp4")

        video_segment = video.subclip(start, end)

        video_segment.write_videofile(output_file, codec="libx264", audio_codec="aac")
        print(f"Segment {video_id}_{segment_id} saved: {output_file}")

    video.close()

print("All video segments have been processed and saved!")

Video file not found for GLd3aX16zBg. Skipping...
Processing video: xHr8X2Wpmno
Moviepy - Building video ./content/video_segments/xHr8X2Wpmno_0.mp4.
MoviePy - Writing audio in xHr8X2Wpmno_0TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./content/video_segments/xHr8X2Wpmno_0.mp4



Moviepy - Done !
Moviepy - video ready ./content/video_segments/xHr8X2Wpmno_0.mp4
Segment xHr8X2Wpmno_0 saved: ./content/video_segments/xHr8X2Wpmno_0.mp4
Moviepy - Building video ./content/video_segments/xHr8X2Wpmno_1.mp4.
MoviePy - Writing audio in xHr8X2Wpmno_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./content/video_segments/xHr8X2Wpmno_1.mp4



Moviepy - Done !
Moviepy - video ready ./content/video_segments/xHr8X2Wpmno_1.mp4
Segment xHr8X2Wpmno_1 saved: ./content/video_segments/xHr8X2Wpmno_1.mp4
Moviepy - Building video ./content/video_segments/xHr8X2Wpmno_2.mp4.
MoviePy - Writing audio in xHr8X2Wpmno_2TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready ./content/video_segments/V53XmPeyjIU_5.mp4
Segment V53XmPeyjIU_5 saved: ./content/video_segments/V53XmPeyjIU_5.mp4
Moviepy - Building video ./content/video_segments/V53XmPeyjIU_6.mp4.
MoviePy - Writing audio in V53XmPeyjIU_6TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./content/video_segments/V53XmPeyjIU_6.mp4



Moviepy - Done !
Moviepy - video ready ./content/video_segments/V53XmPeyjIU_6.mp4
Segment V53XmPeyjIU_6 saved: ./content/video_segments/V53XmPeyjIU_6.mp4
Moviepy - Building video ./content/video_segments/V53XmPeyjIU_7.mp4.
MoviePy - Writing audio in V53XmPeyjIU_7TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ./content/video_segments/V53XmPeyjIU_7.mp4



Moviepy - Done !
Moviepy - video ready ./content/video_segments/V53XmPeyjIU_7.mp4
Segment V53XmPeyjIU_7 saved: ./content/video_segments/V53XmPeyjIU_7.mp4
Video file not found for mZwK0TBI1iY. Skipping...
Video file not found for H_9oM1Y60lU. Skipping...
Video file not found for XOwypmUT5cc. Skipping...
Video file not found for _xOx9hkJoBk. Skipping...
Video file not found for 67ilysu8CeU. Skipping...
Video file not found for gEYyWqs1oL0. Skipping...
Video file not found for nfYzqyureLo. Skipping...
Video file not found for skrWT6xHVoI. Skipping...
Video file not found for BGCW_PWhY9g. Skipping...
Video file not found for EjHxMfOdl1k. Skipping...
Video file not found for MIXkGU4TJp0. Skipping...
Video file not found for Ap3g0IRcjzU. Skipping...
Video file not found for 4AIahLZfNq4. Skipping...
Video file not found for 7nH0Y3agsas. Skipping...
Video file not found for wXA-1ZmvuEA. Skipping...
Video file not found for Kbrkj7wZ4UA. Skipping...
Video file not found for pluBtIfxUvE. Skipping

# Frame extraction

In [4]:
import cv2
import os

def extract_frames_from_all_videos(video_folder, output_folder, interval=2):
    for video_file in os.listdir(video_folder):
        video_path = os.path.join(video_folder, video_file)

        if video_file.endswith('.mp4'):
            video_name = os.path.splitext(video_file)[0]
            video_output_folder = os.path.join(output_folder, video_name)
            os.makedirs(video_output_folder, exist_ok=True)

            extract_frames(video_path, video_output_folder, interval)

def extract_frames(video_path, output_folder, interval=2):
    cap = cv2.VideoCapture(video_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * interval)

    frame_count = 0
    saved_frame_count = 0
    success, frame = cap.read()
    while success:
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1

        success, frame = cap.read()
        frame_count += 1

    cap.release()


video_folder = './content/video_segments'
output_folder = './content/frames'
extract_frames_from_all_videos(video_folder, output_folder, interval=2)

# preprocess caption

In [1]:
!pip install hazm

In [2]:
import pandas as pd
from hazm import Normalizer, word_tokenize, stopwords_list, Lemmatizer

normalizer = Normalizer()
lemmatizer = Lemmatizer()
stopwords = set(stopwords_list())

def preprocess_persian_text(text):
    """
    Preprocess Persian text:
    - Normalize text
    - Tokenize into words
    - Lemmatize verbs
    - Remove stopwords
    """
    normalized_text = normalizer.normalize(text)

    tokens = word_tokenize(normalized_text)

    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords]

    return " ".join(clean_tokens)

def process_and_save_dataset(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    df['cleaned_sentence'] = df['sentence'].apply(preprocess_persian_text)

    df.to_csv(output_csv, index=False)
    print(f"Cleaned dataset saved to {output_csv}")

process_and_save_dataset('./content/translated_training_dataset.csv', 'cleaned_training_dataset.csv')
process_and_save_dataset('./content/translated_validation_dataset.csv', 'cleaned_validation_dataset.csv')

Cleaned dataset saved to cleaned_training_dataset.csv
Cleaned dataset saved to cleaned_validation_dataset.csv


# preprocess frames

In [3]:
!pip install torch torchvision pillow

In [4]:
import os
from PIL import Image
import torch
from torchvision import transforms

clip_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.48145466, 0.4578275, 0.40821073],
        std=[0.26862954, 0.26130258, 0.27577711]
    )
])

def preprocess_frames(input_folder, output_folder):
    """
    Resize and normalize all frames in the input folder structure.
    Save the processed images into the output folder.
    """
    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if os.path.isdir(subfolder_path):
            output_subfolder = os.path.join(output_folder, subfolder)
            os.makedirs(output_subfolder, exist_ok=True)

            for frame_file in os.listdir(subfolder_path):
                if frame_file.endswith('.jpg'):
                    frame_path = os.path.join(subfolder_path, frame_file)
                    output_path = os.path.join(output_subfolder, frame_file)

                    image = Image.open(frame_path).convert('RGB')
                    processed_image = clip_preprocess(image)

                    torch.save(processed_image, output_path.replace('.jpg', '.pt'))
                    print(f"Processed and saved: {output_path.replace('.jpg', '.pt')}")

input_frames_folder = './content/frames'
output_frames_folder = './content/processed_frames'

preprocess_frames(input_frames_folder, output_frames_folder)

Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_0.pt
Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_5.pt
Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_3.pt
Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_1.pt
Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_4.pt
Processed and saved: ./content/processed_frames/V53XmPeyjIU_6/frame_2.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_2/frame_0.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_2/frame_3.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_2/frame_1.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_2/frame_2.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_4/frame_0.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_4/frame_3.pt
Processed and saved: ./content/processed_frames/xHr8X2Wpmno_4/frame_1.pt
Processed and saved: ./content/processed_frames/xHr

# utils

In [ ]:
# delete folder

import shutil

# Path to the folder you want to delete
frames_folder = './content/frames'

# Delete the folder and all its contents
try:
    shutil.rmtree(frames_folder)
    print(f"Deleted folder: {frames_folder}")
except FileNotFoundError:
    print(f"The folder {frames_folder} does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")


Deleted folder: ./content/frames


In [6]:
# zip folder

import os
import zipfile

def zip_frames_folder(source_folder, output_zip):
    # Create a zip file
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through all files and directories within the source folder
        for root, dirs, files in os.walk(source_folder):
            for file in files:
                file_path = os.path.join(root, file)
                # Write the file to the zip archive, preserving the folder structure
                arcname = os.path.relpath(file_path, source_folder)
                zipf.write(file_path, arcname)

    print(f"Frames folder zipped successfully into: {output_zip}")

# Usage
frames_folder = './content/frames'  # Replace with the path to your frames folder
output_zip_file = 'frames.zip'  # Name of the output zip file
zip_frames_folder(frames_folder, output_zip_file)


Frames folder zipped successfully into: frames.zip
